This is a simple notebook that gives a demonstration of k-means, DBSCAN and GMM.

It accompanies Chapter 7 of the book and also shows how different figures were made.

Copyright: Viviana Acquaviva (2023); see also other credits below, in particular Jake Vanderplas' [excellent notebooks](https://jakevdp.github.io/PythonDataScienceHandbook/05.00-machine-learning.html).

Modifications by Julieta Gruszko (2025)

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn import metrics

We'll use a new library, mlxtend, to plot decision regions:

In [ ]:
%pip install mlxtend --upgrade # I had to run this to make plot_decision_regions work

In [ ]:
from mlxtend.plotting import plot_decision_regions

First, we'll make an example data set to try out our clustering approaches, and plot them to take a look.

In [ ]:
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.6, random_state=2)

plt.scatter(X[:, 0], X[:, 1], s =3, c ='gray') # plot original points
plt.xlim(-10,5);


### Question:
- Describe what $\texttt{make\_blobs}$ has generated as our fake data. Of the 3 clustering techniques we discussed (k-means, DBSCAN, GMM), which do you expect to perform well on this data?

Next, we'll try clustering the data with k-means. For now we'll initialize the starting points by hand, just to make sure results are reproducible. We'll plot the results of k-means clustering after convergence.

In [ ]:
fig = plt.figure(figsize=(12,6))

start = np.array([[-1,1],[1,-1],[3,-3],[-5,-10]]) #initial points (fixed for reproducibility; Fig 7.3)

plt.subplot(1,2,1)
plt.scatter(X[:, 0], X[:, 1], s =3, c ='gray') # plot original points
plt.xlim(-10,5);


plt.subplot(1,2,2)
kmeans = KMeans(n_clusters=4, init = "random", n_init=1)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centroids = kmeans.cluster_centers_
plt.scatter(X[:, 0], X[:, 1], s = 3, c = y_kmeans, cmap = 'rainbow') # plot original points
plt.scatter(centroids[:, 0], centroids[:, 1], s=20, edgecolor = 'k', c = [0,1,2,3],cmap = 'rainbow');
plt.xlim(-10,5);
plt.annotate('Converged', xy=(77, 20), xycoords='axes points', size=14, ha='right', va='top')


In [ ]:
print(kmeans.get_params()) # as usual, we can see the parameters of the model
print(kmeans.n_iter_) #we can also print attributes

### Question:
- How many iterations did it take for k-means to converge, in this case? How does that compare to the maximum allowed number of iterations?
- How did we initialize the k-means algorithm, in this case? Change the initialization strategy. Do the results or required number of iterations change?
- What does the $\texttt{n\_init}$ parameter control? When might you want to set it to something other than 1?

In this case, we knew the correct number of clusters becayse we generated the data, but we can use the elbow curve plot to see when the benefit of adding additional clusters "levels off":

In [ ]:
inertiasb = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, n_init = 10)
    kmeans.fit(Xb)
    inertiasb.append(kmeans.inertia_)

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(range(2, 10), inertiasb)
#plt.grid(True)
plt.title('Elbow curve for blobs');
plt.xlabel('Number of clusters $k$', fontsize = 14);
plt.ylabel('$k$-means cost function', fontsize = 14);


In this case, the elbow curve is predicting the correct number of blobs -- the biggest change in slope is at 4 clusters. Notice that there's a pretty big change in slope at 3 as well. That's because 2 of the clusters in our generated distribution are closer together than the others.

### First slightly tricky example: overlapping blobs of different size/density.

In [ ]:
X1b, y1b = make_blobs(n_samples=200, centers=[(1.25,1)],
                       cluster_std=0.2, random_state=1)

X2b, y2b = make_blobs(n_samples=400, centers=[(0,1)],
                       cluster_std=0.5, random_state=2)

X3b, y3b = make_blobs(n_samples=200, centers=[(-1.25,1)],
                       cluster_std=0.2, random_state=3)

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.scatter(X1b[:, 0], X1b[:, 1], s =10, c ='gray') # plot original points

plt.scatter(X2b[:, 0], X2b[:, 1], s =10, c ='violet') # plot original points

plt.scatter(X3b[:, 0], X3b[:, 1], s =10, c ='teal') # plot original points

Right now, this is labeled data! Let's pretend this is an unsupervised learning problem by combining clusters in single data set, but we'll make a set of the original labels for reference.

In [ ]:
Xb = np.vstack([X1b,X2b,X3b])
yb = np.concatenate([np.zeros(len(y1b)),np.zeros(len(y2b))+1,np.zeros(len(y3b))+2])

Fixing the random seed generates consistent predictions (the predictions of the cluster assignments are usually consistent, but which cluster is predicted as 0, which as 1, which as 2 would otherwise vary.)

In [ ]:
kmeans = KMeans(n_clusters=3, n_init = 10, random_state=30) #predicts 0,1,2
kmeans.fit(Xb)
yb_kmeans = kmeans.predict(Xb)
centersb = kmeans.cluster_centers_

In [ ]:
yb_kmeans

Now we can compare the cluster assignments from the k means algorithm to the original ones. This isn't something we can normally do with unsupervised learning problems! Notice that k-means has commuted the category labels (colors), but that's ok.

In [ ]:
plt.figure(figsize=(8,6))
model = KMeans(n_clusters=3, n_init = 10, random_state=30) #predicts 0,1,2
model.fit(Xb)
plot_decision_regions(Xb, yb.astype(int), clf = model, legend=0, markers = '...', colors = 'teal,violet,lightgray')
plt.scatter(centersb[:, 0], centersb[:, 1], c='black', s=100, alpha=0.5);

plt.xlim(-2.5,2.5)
plt.ylim(-0.5,2.5);

### Question:
- Has k-means performed well on this data? Do you expext improved performance from using either DBSCAN or GMM?

### Now we move on to a different distribution (smiley face).

Generate points for the smiley face distribution:

In [ ]:
from math import pi, cos, sin
from random import random

def point(h, k, r):
    theta = random() * 2 * pi
    return h + cos(theta) * r, k + sin(theta) * r + 0.2*random()

xy = [point(1,2,1) for _ in range(100)]

In [ ]:
X1, y1 = make_blobs(n_samples=10, centers=[(0.5,2.5)],
                       cluster_std=0.05, random_state=1)

X2, y2 = make_blobs(n_samples=10, centers=[(1.5,2.5)],
                       cluster_std=0.05, random_state=2)

X3, y3 = make_blobs(n_samples=10, centers=[(1,1.7)],
                       cluster_std=0.05, random_state=2)

In [ ]:
X3_stretch = np.array([X3[:,0]*3, X3[:,1]]) #make the mouth :) 

In [ ]:
plt.axes().set_aspect('equal', 'datalim')
plt.scatter(*zip(*xy))
plt.scatter(X1[:,0],X1[:,1])
plt.scatter(X2[:,0],X2[:,1])
plt.scatter(X3_stretch.T[:,0]-1.9,X3_stretch.T[:,1])

plt.show()

Finally, put together the four sets of points in the figure to create the data set as a single array:


In [ ]:
X = np.vstack([xy,X1,X2,np.array([X3_stretch.T[:,0]-1.9,X3_stretch.T[:,1]]).T])

### Let's see how k-means clusters these points.

In [ ]:
kmeans = KMeans(n_clusters=4, n_init = 10, random_state=32) #We can also change the number of clusters
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=10, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.5);

Create vector with true labels (color the points):

In [ ]:
y = np.concatenate([np.zeros(len(xy)), np.zeros(len(y1))+1,np.zeros(len(y2))+2,np.zeros(len(y3))+3])

Finally, compare the clustering assignments of k means (background color) to the true labels:

In [ ]:
plt.figure(figsize=(8,6))
plot_decision_regions(X, y.astype(int), clf = kmeans, legend=0, markers = '.', colors = 'lightgray,teal,yellow,violet')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.5);
plt.scatter(*zip(*xy), s = 30, c = 'lightgray', edgecolors='k')
plt.scatter(X1[:,0],X1[:,1], s = 30, c = 'teal',edgecolors='k')
plt.scatter(X2[:,0],X2[:,1], s = 30, c = 'yellow', edgecolors='k')
plt.scatter(X3_stretch.T[:,0]-1.9,X3_stretch.T[:,1],s = 30, c = 'violet', edgecolors='k')
plt.xlim(-0.5,2.5);
plt.ylim(0.5,3.5);
#plt.savefig('ClustersBad2.pdf', dpi = 300)

This isn't very good! Let's see if optimizing the number of clusters using the elbow curve helps. 

In [ ]:
inertias = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, n_init = 10)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(range(2, 10), inertias)
plt.title('Elbow curve for smiley face');
plt.xlabel('Number of clusters $k$', fontsize = 14);
plt.ylabel('$k$-means cost function', fontsize = 14);


Again, n = 4 is the preferred choice: this is correct but mostly by chance, as the distribution is not one that k-means can handle correctly.

### Silhouette score

The silhouette score is another metric used to evaluate the success of a clustering method and pick a number of clusters. Higher values correspond to better-defined clustering schemes.

Just remember, silhouette scores rely on comparing intra-cluster vs. inter-cluster distances, so they won't give nice-looking results for "closely spaced spaghetti" or "nested arc"-like clusters! Alternatives that are better for those types of distributions do exist: you can check out "Density-Based Clustering Validation (DBCV)" as one example, though it's not built in to sk-learn.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
#Smiley face

n_clusters = [2,3,4,6]

for n in n_clusters:
    
    model = KMeans(n_clusters = n, n_init = 10, random_state=10)

    model.fit(X)

    y_kmeans = model.predict(X)

    silhouette_scores = silhouette_samples(X, y_kmeans)

    xlower = 10

    fig, axs = plt.subplots(1, 2, figsize=(16, 8))
    
    ax = axs[1]
    colors = plt.cm.Accent(y_kmeans.astype(float)/n)
    ax.scatter(X[:, 0], X[:, 1], c=colors, s=40, cmap='flare', edgecolor='k');
    ax.tick_params(axis='both', which='both', labelsize=20);

    ax = axs[0]

    for i in np.unique(y_kmeans):
        ind = y_kmeans==i
        silh = np.sort(silhouette_scores[ind])
        size_cluster_i = silh.shape[0]
        xupper = xlower + size_cluster_i
        color = plt.cm.Accent(float(i)/model.n_clusters)
        ax.fill_between(np.arange(xlower, xupper), 0, silh, facecolor=color, edgecolor=color, alpha=0.7)
        ax.axhline(y=0, c='k', lw=2)
        ax.text(0.05, 0.95, '%0.0f clusters'%n, transform=ax.transAxes, fontsize=20)
        ax.text(0.45, 0.95, 'Mean S. score: %0.2f'%np.mean(silhouette_scores), transform=ax.transAxes, fontsize=20)
        xlower = xupper + 10
        ax.set_ylabel('Silhouette score', fontsize=16)
        ax.set_ylim(-0.2,0.8)
        
    ax.axhline(y=np.mean(silhouette_scores), color="red", linestyle="--")
    ax.tick_params(axis='both', which='both', labelsize=20);
    ax.set_xticks([]);
#    figname = 'SilhouetteSmiley'+str(n)+'.pdf'
#    plt.savefig(figname, dpi = 300)

In the example above, the silhouette score points to 4 clusters, but keep in mind that any of these methods (elbow method, silhouette score) only makes sense if the hypothesis that clustered can be correctly identified by k-means (in other words, are convex, globular, and similar in shape and density). This is not the case here! That's also why we see a higher average silhouette score the more clusters we add.

### We can now take a quick look at two density-based clustering methods, DBSCAN and OPTICS.

In [ ]:
from sklearn.cluster import DBSCAN

#Code adapted from: https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=0.25, min_samples=2).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

We can see how the clustering changes as we vary the distance parameter, eps:

In [ ]:
# #############################################################################
# Compute DBSCAN

for i,eps in enumerate([0.2, 0.25, 0.3, 0.35]):
    
    plt.figure(figsize = (6,6))
    
    db = DBSCAN(eps=eps, min_samples=2).fit(X)

    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)

# #############################################################################


# Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
        # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=10)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

    plt.title('$\\epsilon$ = %0.2f; estimated number of clusters: %d' % (eps, n_clusters_))
    
    plt.savefig('DBSCAN_'+str(i)+'.pdf', dpi = 300)
    
    

### Questions:
- How does DBSCAN perform for this distribution? 
- What does the epsilon parameter control?

My final note here is that building and evaluating clustering schemes is tricky. For our clustering to make sense, we need to have a good knowledge of the structure of our data, but unfortunately, this is often what we'd like to learn by building the clustering in first place. I don't have a lot of wisdom to share; just this word of caution. On Thursday we'll discuss dimensionality reduction methods, which you can use (with caution!) to visualize the structure of your data, which can help you choose a clustering method.

### Finally, we will take a look at how to use a probabilistic model: Gaussian Mixture Models.

Again from sklearn docs: A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. One can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians.

In [ ]:
from sklearn import mixture

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
#Function from https://jakevdp.github.io/PythonDataScienceHandbook/06.00-figure-code.html#Covariance-Type

def draw_ellipse(gmm, ax, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    for n in range(gmm.n_components):
        if gmm.covariance_type == 'full':
            covariances = gmm.covariances_[n]
        elif gmm.covariance_type == 'tied':
            covariances = gmm.covariances_
        elif gmm.covariance_type == 'diag':
            covariances = np.diag(gmm.covariances_[n])
        elif gmm.covariance_type == 'spherical':
            covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        
        # Draw the Ellipse
        for nsig in range(1, 4): #1, 2, and 3 sigma
            ell = Ellipse(gmm.means_[n], nsig *v[0], nsig *v[1], angle = angle, **kwargs)
            ax.add_patch(ell)


Just a demonstration of what the different covariance options look like:

In [ ]:
#Figure also adapted from https://jakevdp.github.io/PythonDataScienceHandbook/06.00-figure-code.html#Covariance-Type
 
fig, ax = plt.subplots(1, 3, figsize=(14, 4), sharey=True)
fig.subplots_adjust(wspace=0.05)

rng = np.random.RandomState(20)
Xe = np.dot(rng.randn(500, 2), rng.randn(2, 2))

for i, cov_type in enumerate(['full','diag', 'spherical']):
    model = mixture.GaussianMixture(n_components=1, covariance_type=cov_type).fit(Xe)
    ax[i].axis('equal')
    ax[i].scatter(Xe[:, 0], Xe[:, 1], edgecolor='k', alpha=0.5)
    ax[i].set_title('covariance_type="{0}"'.format(cov_type), size=14, family='monospace')
    draw_ellipse(gmm=model, ax=ax[i], alpha=0.1, edgecolor='k', facecolor='#808080')
    ax[i].xaxis.set_major_formatter(plt.NullFormatter())
    ax[i].yaxis.set_major_formatter(plt.NullFormatter())
    ax[i].set_xlim(-5, 5)
    

### Question:
- In the plots above, what is being shown by the contour lines?
- What is the difference between the "full," "diag," and "spherical" covariance options, in terms of the symmetry and "stretching" of the Gaussians used for clustering?


### Modeling with a Gaussian Mixture Model predicts probabilities.

We'll start with the 3 blob example from above.

In [ ]:
model = mixture.GaussianMixture(n_components=3, covariance_type='full',random_state=30) 

model.fit(Xb)

y_GMM = model.predict(Xb)

probs = model.predict_proba(Xb)

size = 50 * probs.max(axis=1)**4

In [ ]:
y_GMM # 

### GMM decision function for a "full" covariance:

In [ ]:
model = mixture.GaussianMixture(n_components=3, covariance_type='full',random_state=41)

model.fit(Xb)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

plt.axis('equal')

plot_decision_regions(Xb, yb.astype(int), 
        clf=model, legend=0, markers = '...', colors = 'lightgray,violet,teal')

ax.set_title('covariance_type="{0}"'.format('full'), size=14, family='monospace')

ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-0.5,2.5)

#plt.savefig('GMM_blobs_full.pdf', dpi = 300)

As before, the labels are permuted, but that's fine. 

### Now, for comparison, with a spherical covariance:

In [ ]:
model = mixture.GaussianMixture(n_components=3, covariance_type='spherical',random_state=41)

model.fit(Xb)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

plt.axis('equal')

plot_decision_regions(Xb, yb.astype(int), 
        clf=model, legend=0, markers = '...', colors = 'lightgray,violet,teal')

ax.set_title('covariance_type="{0}"'.format('spherical'), size=14, family='monospace')

ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-0.5,2.5)

#plt.savefig('GMM_blobs_spherical.pdf', dpi = 300)

### Question:
- Which performs better for this problem: k-means or GMM?

### Finally, we can apply GMMs to the smiley face problem.

We start with 4 components.
### Question:
- Do you expect this to work? Why or why not? Consider the assumption(s) built in to the GMM method.

In [ ]:
gmm4 = mixture.GaussianMixture(n_components=4, covariance_type='full', random_state=1)

gmm4.fit(X)

plt.figure(figsize=(8,6))

xy = [point(1,2,1) for _ in range(100)]

plot_decision_regions(X, y.astype(int), 
        clf=gmm4, legend=0, markers = '.', colors = 'lightgray,yellow,teal,violet')

plt.xlim(-0.5,2.5);

plt.ylim(0.5,3.5);

#plt.savefig('GMMbad.pdf', dpi = 300)

### We can use the BIC criterion to figure out how many Gaussian mixture components best fit the smiley face in the GMM model.

Note that results tend to vary slightly between different runs; in the book we show a run for 10 components, but obtaining 9 or 11 (or even a second minimum at 13) is not uncommon!

In [ ]:
n_components = np.arange(1, 30)

models = [mixture.GaussianMixture(n, covariance_type='full', random_state=10).fit(X)
          for n in n_components]

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
ax.plot(n_components, [m.bic(X) for m in models], label='BIC')
ax.legend(loc='best', fontsize=20)
ax.set_xlabel('n_components', fontsize=20);
ax.tick_params(axis='both', which='both', labelsize=20);
#plt.savefig('GMM_smiley_BIC.pdf', dpi = 300)

### And re-do the plot with the appropriate number of components.

In [ ]:
#These two functions (note that "draw_ellipse" is not the same as before!) are also from Jake Vanderplas' notebooks.

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0.0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, width = nsig * width, height = nsig * height, angle=angle, **kwargs))

def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='Accent', zorder=2, edgecolor='k')
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2, edgecolor='k')
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, facecolor='#808080', edgecolor='k', alpha=w * w_factor)
        
    ax.tick_params(axis='both', which='both', labelsize=20);

In [ ]:
gmm10 = mixture.GaussianMixture(n_components=10, covariance_type='full', random_state=0)

fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(111, aspect='equal')

plot_gmm(gmm10, X, label=False, ax=ax)

plt.xlim(-0.5,2.5);

plt.ylim(0.5,3.5);

plt.text(-0.3,0.7,'Original', fontsize = 18)

ax.tick_params(axis='both', which='both', labelsize=20);

#plt.savefig('Smiley_GMM_10.pdf', dpi = 300)


### Question:
- How does the classification approach taken by GMM differ from those taken by k-means and DBSCAN? Does each point correspond to just a single cluster, in this case?

### Finally, we can use our GMM with 10 components as a generative model to generate new samples that follow the smiley face distribution!

In [ ]:
Xnew = gmm10.sample(n_samples=500) #This is how we generate new samples!

fig = plt.figure(figsize=(8, 8))

ax2 = fig.add_subplot(111, aspect='equal', sharey = ax)

ax2.scatter(Xnew[0][:, 0], Xnew[0][:, 1], s = 40, facecolor='r', edgecolor='k', alpha=0.5);

ax2.tick_params(axis='both', which='both', labelsize=20);

plt.xlim(-0.5,2.5);

plt.ylim(0.5,3.5);

plt.text(-0.3,0.7,'Generated', fontsize = 18)

#plt.savefig('Smiley_GMM_generated.pdf', dpi = 300)

Not bad, but also not perfect! Since our GMM clustering wasn't able to separate the right eye from the outline of the smiley face into two separate clusters, the generated data will reproduce this error. Something to be careful about with generative AI methods!

### Next, move on to testing clustering for galaxy classification in the $\texttt{Studio10\_GalaxyClustering.ipynb}$ notebook.

### Acknowledgement statement:

Upload both notebooks to Gradescope for this week!